In [1]:
import dgipy

## Get Drug-Gene Interaction (DGI) Data
Use DGIpy to query DGIdb for DGI data and drug applications for drugs of interest. The appNo's will be used to pull relevant FDA labels from the JSON dataset. The DGI data will eventually be linked up with whatever is extracted from the JSON dataset. The drugs of interest for this experiment are:  
  
- **Imatinib** (favorite test case)
- **Encorafenib** (complex targeted therapy)
- **Cisplatin** (common cancer therapy)
- **Aspirin** (common therapy)
- **FAKE** (fake)

In [2]:
drugs = ['imatinib','encorafenib','cisplatin','aspirin','fake']
dgi_data = dgipy.get_interactions(drugs, search='drugs')
dgi_data

,drug,gene,approval,score,interaction_attributes,source,pmid
0,IMATINIB,FIP1L1,True,2.199452,Fusion Protein: FIP1L1:PDGFRA,FDA Pharmacogenomic Biomarkers | PharmGKB - Th...,24433361
1,IMATINIB,ABL1,True,0.119476,Mechanism of Action: Inhibition| Endogenous Dr...,The ChEMBL Bioactivity Database | Drug Target ...,
2,IMATINIB,BIRC5,True,0.016854,,NCI Cancer Gene Index,16254145
3,IMATINIB,ULK3,True,0.733151,,PharmGKB - The Pharmacogenomics Knowledgebase,15073101
4,IMATINIB,NF1,True,0.031876,Alteration: NF1:.| Alteration: NF1:del,Cancer Genome Interpreter,
...,...,...,...,...,...,...,...
296,CISPLATIN,MPHOSPH8,True,0.003043,,Drug Target Commons,
297,CISPLATIN,MDM2,True,0.013811,Alteration: MDM2:amp,Cancer Genome Interpreter,
298,CISPLATIN,MLLT3,True,0.179547,,PharmGKB - The Pharmacogenomics Knowledgebase,
299,CISPLATIN,MAPT,True,0.000814,,Drug Target Commons,


In [3]:
drug_apps = dgipy.get_drug_applications(drugs)
drug_apps

,drug,application,description
0,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...
1,IMATINIB,ANDA079179,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
2,IMATINIB,ANDA204285,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
3,IMATINIB,ANDA204644,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...
4,IMATINIB,ANDA206547,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
5,IMATINIB,ANDA207495,IMATINIB MESYLATE: EQ 100MG BASE Discontinued ...
6,IMATINIB,ANDA207586,none
7,IMATINIB,ANDA208302,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
8,IMATINIB,ANDA208429,none
9,IMATINIB,ANDA210658,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...


## Load FDA Data, Filter Search Space
Load previously downloaded FDA data set (JSON format). Define search space using the previously obtained ANDA/NDA applications via DGIpy. Filter the FDA data set using this search space definition. NOTE: I think it will be important to maintain the *brand_name* and *application_number* sections throughout any downstream data manipulation to maintain the ability to somehow link it back to paired DGI data.  
  
I also think this method would be used to obtain search chunks for any question answering or generative tasks that may be desirable (ala RAG technique that James presented).

In [4]:
# from datasets import Dataset
import pandas as pd
drugs_at_fda = pd.read_excel('../data/openfda-2.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)

relevant_labels = list(set(drug_apps['application']))
search_set = drugs_at_fda[drugs_at_fda['application_number'].isin(relevant_labels)].reset_index(drop=True)
search_set

,brand_name,application_number,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,Imatinib,ANDA206547,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None . None ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
1,Cisplatin,ANDA207323,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNINGS Cisplatin should be administered unde...,NaN,NaN
2,Imatinib Mesylate,ANDA207495,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. (4),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
3,Imatinib Mesylate,ANDA204644,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
4,Imatinib Mesylate,ANDA207586,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
5,CISplatin,ANDA075036,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNING Cisplatin Injection should be administ...,NaN,NaN
6,VAZALORE,NDA203697,NaN,Uses Uses For temporary relief of minor aches ...,NaN,NaN,WARNINGS Warnings Reye’s Syndrome: Children an...,NaN,NaN,Active Ingredient (in Each capsules) Purposes:...,NaN,ACTIVE iNGREDIENT Active Ingredient (in Each c...,Do not use I f you have ever had an allergic r...,NaN,NaN,NaN
7,Imatinib Mesylate,ANDA079179,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None ( 4 ).,5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
8,BRAFTOVI,NDA210496,6 ADVERSE REACTIONS The following adverse reac...,1 INDICATIONS AND USAGE BRAFTOVI is a kinase i...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • New Primary Malig...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Cardiac Electrophysiolog...,NaN
9,imatinib mesylate,ANDA212135,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN


## NER Pipeline Methods

Use previously written methods from huggingface ecosystem to quickly perform NER for diseases, genes, and chemical entities across search set. Models used are out of the Escuela Tecnica Superior (ETS) Universidad Politecnica de Madrid and are versions of BioBERT that have been fine-tuned for instances of genetico, quimicos, and enfermedades. For each model, two additional corpi were used to fine-tune each entity class:  
  
- Diseases (BC5CDR - Diseases, NCBI - Diseases)
- Chemicals (BC4CHEMD, BC5CDR - Chemicals)
- Genes/Proteins (JNLPBA, BC2GM)
  
Further documentation can be found from: 
  
- Original dissertation: https://oa.upm.es/67933/1/TFM_ALVARO_ALONSO_CASERO.pdf 
- Repo: https://github.com/librairy/bio-ner
- Huggingface:
    - Genetics: https://huggingface.co/alvaroalon2/biobert_genetic_ner  
    - Diseases: https://huggingface.co/alvaroalon2/biobert_diseases_ner
    - Chemicals: https://huggingface.co/alvaroalon2/biobert_chemical_ner

In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

model_checkpoint = "alvaroalon2/biobert_genetic_ner"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pipe_gene = pipeline("token-classification", model="alvaroalon2/biobert_genetic_ner",aggregation_strategy='first')
pipe_chemical = pipeline("token-classification", model="alvaroalon2/biobert_chemical_ner",aggregation_strategy='first')
pipe_disease = pipeline("token-classification", model="alvaroalon2/biobert_diseases_ner",aggregation_strategy='first')


def pre_process(label_header,df):
    def get_tokens(entry):
        return {'tokens': tokenizer(entry[label_header]).tokens() }

    f = pd.DataFrame(df, columns=[label_header,'application_number'])
    dataset = Dataset.from_pandas(f)

    print(f'UNFILTERED: {len(dataset[label_header])}')

    dataset = dataset.filter(lambda x: x[label_header] is not None)
    dataset = dataset.map(get_tokens)
    print(f'After NONE CHECK: {len(dataset[label_header])}')

    return(dataset)


def pipe(label_header,dataset):
    def generate_genomic_ner(entry):
        return {'genomic_ner': pipe_gene(entry[label_header]) }

    def generate_chemical_ner(entry):
        return {'chemical_ner': pipe_chemical(entry[label_header]) }

    def generate_diseases_ner(entry):
        return {'diseases_ner': pipe_disease(entry[label_header]) }

    def merged_ner_record(entry):
        return {'merged_ner_groups': entry['genomic_ner'] + entry['chemical_ner'] + entry['diseases_ner']}

    dataset = dataset.map(generate_genomic_ner)
    dataset = dataset.map(generate_chemical_ner)
    dataset = dataset.map(generate_diseases_ner)

    dataset = dataset.map(merged_ner_record)

    return(dataset)


def build_dataframe(feature,dataset):

    # Build DF
    df = pd.DataFrame()

    for entry in dataset[feature]:
        tdf = pd.DataFrame(entry)
        df = pd.concat([df,tdf]).reset_index(drop=True)

    return(df)

2024-02-06 09:18:00.928260: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Apply pipeline methods to label sections of interest. Sections of interest and desired entities are:  
  
- Indications -> extracted entities point to approved use case and may contain exact disease states or genetics that are relevant for the interaction context
- Pharmacogenomics -> extracted entities may contain genetics and enzymatic properties that are relevant for the interaction context
- Clinical Pharmacology -> extracted entites may contain genetics, enzymatic information, or other situations that are relevant for the interaction context
  
NOTE: Not currently applied, post-processing methods are in progress that can align and evaluate entities against original tokenization.

### Indications

In [6]:
indications = pre_process('indications_and_usage',search_set)
indications = pipe('indications_and_usage',indications)

UNFILTERED: 39


Filter:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1088 > 512). Running this sequence through the model will result in indexing errors


After NONE CHECK: 38


Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

### Pharmacogenomics

In [7]:
pharmacogenomics = pre_process('pharmacogenomics',search_set)
pharmacogenomics = pipe('pharmacogenomics',pharmacogenomics)

UNFILTERED: 39


Filter:   0%|          | 0/39 [00:00<?, ? examples/s]

After NONE CHECK: 0


### Clinical Pharmacology

In [8]:
clinical_pharmacology = pre_process('clinical_pharmacology',search_set)
clinical_pharmacology = pipe('clinical_pharmacology',clinical_pharmacology)

UNFILTERED: 39


Filter:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

After NONE CHECK: 36


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

## Inspect 
Inspect the data and see whats up.

In [9]:
# Finding 1: No pharmacogenomics sections from this search set
pharmacogenomics

Dataset({
    features: ['pharmacogenomics', 'application_number'],
    num_rows: 0
})

In [10]:
# Finding 2: Maybe its good at just pulling indications?
# but it also likes to pull type 0 classifications
# https://huggingface.co/docs/transformers/v4.19.2/en/tasks/token_classification

print(indications['indications_and_usage'][0])

1 INDICATIONS AND USAGE Imatinib mesylate is a kinase inhibitor indicated for the treatment of: Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+CML) in chronic phase (1.1) Patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in blast crisis (BC), accelerated phase (AP), or in chronic phase (CP) after failure of interferon-alpha therapy (1.2) Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) (1.3) Pediatric patients with newly diagnosed Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) in combination with chemotherapy (1.4) Adult patients with myelodysplastic/myeloproliferative diseases (MDS/MPD) associated with platelet-derived growth factor receptor (PDGFR) gene re-arrangements (1.5) Adult patients with aggressive systemic mastocytosis (ASM) without the D816V c-Kit mutation or with c-Kit mutational status un

1 INDICATIONS AND USAGE Imatinib mesylate is a kinase inhibitor indicated for the treatment of: Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+CML) in chronic phase (1.1) Patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in blast crisis (BC), accelerated phase (AP), or in chronic phase (CP) after failure of interferon-alpha therapy (1.2) Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) (1.3) Pediatric patients with newly diagnosed Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) in combination with chemotherapy (1.4) Adult patients with myelodysplastic/myeloproliferative diseases (MDS/MPD) associated with platelet-derived growth factor receptor (PDGFR) gene re-arrangements (1.5) Adult patients with aggressive systemic mastocytosis (ASM) without the D816V c-Kit mutation or with c-Kit mutational status unknown (1.6) Adult patients with hypereosinophilic syndrome (HES) and/or chronic eosinophilic leukemia (CEL) who have the FIP1L1-PDGFRα fusion kinase (mutational analysis or fluorescence in situ hybridization [FISH] demonstration of CHIC2 allele deletion) and for patients with HES and/or CEL who are FIP1L1-PDGFRα fusion kinase negative or unknown (1.7) Adult patients with unresectable, recurrent and/or metastatic dermatofibrosarcoma protuberans (DFSP) (1.8) Patients with Kit (CD117) positive unresectable and/or metastatic malignant gastrointestinal stromal tumors (GIST) ( 1.9 ) Adjuvant treatment of adult patients following resection of Kit (CD117) positive GIST ( 1.10 ) 1.1 Newly Diagnosed Philadelphia Positive Chronic Myeloid Leukemia (Ph+ CML) Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+CML) in chronic phase. 1.2 Ph+ CML in Blast Crisis (BC), Accelerated Phase (AP) or Chronic Phase (CP) After Interferon-alpha (IFN) Therapy Patients with Philadelphia chromosome positive chronic myeloid leukemia in blast crisis, accelerated phase, or in chronic phase after failure of interferon-alpha therapy. 1.3 Adult Patients With Ph+ Acute Lymphoblastic Leukemia (ALL) Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ALL). 1.4 Pediatric Patients With Ph+ Acute Lymphoblastic Leukemia (ALL) Pediatric patients with newly diagnosed Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) in combination with chemotherapy. 1.5 Myelodysplastic/Myeloproliferative Diseases (MDS/MPD) Adult patients with myelodysplastic/myeloproliferative diseases associated with platelet-derived growth factor receptor (PDGFR) gene re-arrangements. 1.6 Aggressive Systemic Mastocytosis (ASM) Adult patients with aggressive systemic mastocytosis without the D816V c-Kit mutation or with c-Kit mutational status unknown. 1.7 Hypereosinophilic Syndrome (HES) and/or Chronic Eosinophilic Leukemia (CEL) Adult patients with hypereosinophilic syndrome and/or chronic eosinophilic leukemia who have the FIP1L1-PDGFRα fusion kinase (mutational analysis or fluorescence in situ hybridization [FISH] demonstration of CHIC2 allele deletion) and for patients with HES and/or CEL who are FIP1L1-PDGFRα fusion kinase negative or unknown. 1.8 Dermatofibrosarcoma Protuberans (DFSP) Adult patients with unresectable, recurrent and/or metastatic dermatofibrosarcoma protuberans. 1.9 Kit+ Gastrointestinal Stromal Tumors (GIST) Patients with Kit (CD117) positive unresectable and/or metastatic malignant gastrointestinal stromal tumors. 1.10 Adjuvant Treatment of GIST Adjuvant treatment of adult patients following complete gross resection of Kit (CD117) positive GIST.

In [11]:
# Finding 2: Maybe its good at just pulling indications?
# but it also likes to pull type 0 classifications
# https://huggingface.co/docs/transformers/v4.19.2/en/tasks/token_classification

indications['diseases_ner'][0]

[{'end': 178,
  'entity_group': '0',
  'score': 0.9999987483024597,
  'start': 0,
  'word': '1 INDICATIONS AND USAGE Imatinib mesylate is a kinase inhibitor indicated for the treatment of : Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive'},
 {'end': 203,
  'entity_group': 'DISEASE',
  'score': 0.9999694228172302,
  'start': 179,
  'word': 'chronic myeloid leukemia'},
 {'end': 208,
  'entity_group': '0',
  'score': 0.9999868273735046,
  'start': 204,
  'word': '( Ph +'},
 {'end': 211,
  'entity_group': 'DISEASE',
  'score': 0.9999761581420898,
  'start': 208,
  'word': 'CML'},
 {'end': 282,
  'entity_group': '0',
  'score': 0.9999980330467224,
  'start': 211,
  'word': ') in chronic phase ( 1. 1 ) Patients with Philadelphia chromosome positive'},
 {'end': 307,
  'entity_group': 'DISEASE',
  'score': 0.999965488910675,
  'start': 283,
  'word': 'chronic myeloid leukemia'},
 {'end': 312,
  'entity_group': '0',
  'score': 0.999991238117218,
  'start': 308

In [12]:
# Finding #3: Not sure the best information to pull from this section, nor how. Complex mix of information that might be best pulled using some kind of question answering?
clinical_pharmacology['clinical_pharmacology'][3]

'12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Action Imatinib mesylate is a protein-tyrosine kinase inhibitor that inhibits the BCR-ABL tyrosine kinase, the constitutive abnormal tyrosine kinase created by the Philadelphia chromosome abnormality in CML. Imatinib inhibits proliferation and induces apoptosis in BCR-ABL positive cell lines as well as fresh leukemic cells from Philadelphia chromosome positive chronic myeloid leukemia. Imatinib inhibits colony formation in assays using ex vivo peripheral blood and bone marrow samples from CML patients. In vivo , imatinib inhibits tumor growth of BCR-ABL transfected murine myeloid cells as well as BCR-ABL positive leukemia lines derived from CML patients in blast crisis. Imatinib is also an inhibitor of the receptor tyrosine kinases for platelet-derived growth factor (PDGF) and stem cell factor (SCF), c-Kit, and inhibits PDGF- and SCF-mediated cellular events. In vitro , imatinib inhibits proliferation and induces apoptosis in GIST cells, whic

'12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Action Imatinib mesylate is a protein-tyrosine kinase inhibitor that inhibits the BCR-ABL tyrosine kinase, the constitutive abnormal tyrosine kinase created by the Philadelphia chromosome abnormality in CML. Imatinib inhibits proliferation and induces apoptosis in BCR-ABL positive cell lines as well as fresh leukemic cells from Philadelphia chromosome positive chronic myeloid leukemia. Imatinib inhibits colony formation in assays using ex vivo peripheral blood and bone marrow samples from CML patients. In vivo , imatinib inhibits tumor growth of BCR-ABL transfected murine myeloid cells as well as BCR-ABL positive leukemia lines derived from CML patients in blast crisis. Imatinib is also an inhibitor of the receptor tyrosine kinases for platelet-derived growth factor (PDGF) and stem cell factor (SCF), c-Kit, and inhibits PDGF- and SCF-mediated cellular events. In vitro , imatinib inhibits proliferation and induces apoptosis in GIST cells, which express an activating c-Kit mutation. 12.3 Pharmacokinetics The pharmacokinetics of imatinib mesylate tablets have been evaluated in studies in healthy subjects and in population pharmacokinetic studies in over 900 patients. The pharmacokinetics of imatinib mesylate tablets are similar in CML and GIST patients. Absorption and Distribution Imatinib is well absorbed after oral administration with Cmax achieved within 2-4 hours post-dose. Mean absolute bioavailability is 98%. Mean imatinib AUC increases proportionally with increasing doses ranging from 25 mg to 1,000 mg. There is no significant change in the pharmacokinetics of imatinib on repeated dosing, and accumulation is 1.5- to 2.5-fold at steady state when imatinib mesylate tablets are dosed once daily. At clinically relevant concentrations of imatinib, binding to plasma proteins in in vitro experiments is approximately 95%, mostly to albumin and α1-acid glycoprotein. Elimination Metabolism CYP3A4 is the major enzyme responsible for metabolism of imatinib. Other cytochrome P450 enzymes, such as CYP1A2, CYP2D6, CYP2C9, and CYP2C19, play a minor role in its metabolism. The main circulating active metabolite in humans is the N-demethylated piperazine derivative, formed predominantly by CYP3A4. It shows in vitro potency similar to the parent imatinib. The plasma AUC for this metabolite is about 15% of the AUC for imatinib. The plasma protein binding of N-demethylated metabolite CGP74588 is similar to that of the parent compound. Human liver microsome studies demonstrated that imatinib mesylate tablets are a potent competitive inhibitor of CYP2C9, CYP2D6, and CYP3A4/5 with K i values of 27, 7.5, and 8 μM, respectively. Excretion Imatinib elimination is predominately in the feces, mostly as metabolites. Based on the recovery of compound(s) after an oral 14C-labeled dose of imatinib, approximately 81% of the dose was eliminated within 7 days, in feces (68% of dose) and urine (13% of dose). Unchanged imatinib accounted for 25% of the dose (5% urine, 20% feces), the remainder being metabolites. Following oral administration in healthy volunteers, the elimination half-lives of imatinib and its major active metabolite, the N-demethyl derivative (CGP74588), are approximately 18 and 40 hours, respectively. Typically, clearance of imatinib in a 50-year-old patient weighing 50 kg is expected to be 8 L/h, while for a 50-year-old patient weighing 100 kg the clearance will increase to 14 L/h. The inter-patient variability of 40% in clearance does not warrant initial dose adjustment based on body weight and/or age but indicates the need for close monitoring for treatment-related toxicity. Specific Populations Hepatic Impairment The effect of hepatic impairment on the pharmacokinetics of both imatinib and its major metabolite, CGP74588, was assessed in 84 patients with cancer and varying degrees of hepatic impairment [see Use in Specific Populations (8.6) ] at imatinib doses ranging from 100 mg to 800 mg. Exposure to both imatinib and CGP74588 was comparable between each of the mildly and moderately hepatically-impaired groups and the normal group. Patients with severe hepatic impairment tend to have higher exposure to both imatinib and its metabolite than patients with normal hepatic function. At steady state, the mean C max /dose and AUC/dose for imatinib increased by about 63% and 45%, respectively, in patients with severe hepatic impairment compared to patients with normal hepatic function. The mean C max /dose and AUC/dose for CGP74588 increased by about 56% and 55%, respectively, in patients with severe hepatic impairment compared to patients with normal hepatic function. Dose reductions are necessary for patients with severe hepatic impairment [see Dosage and Administration (2.12) ] . Renal Impairment The effect of renal impairment on the pharmacokinetics of imatinib was assessed in 59 cancer patients with varying degrees of renal impairment [see Use in Specific Populations (8.7)] at single and steady state imatinib doses ranging from 100 to 800 mg/day. The mean exposure to imatinib (dose normalized AUC) in patients with mild and moderate renal impairment increased 1.5- to 2-fold compared to patients with normal renal function. The AUCs did not increase for doses greater than 600 mg in patients with mild renal impairment. The AUCs did not increase for doses greater than 400 mg in patients with moderate renal impairment. Two patients with severe renal impairment were dosed with 100 mg/day and their exposures were similar to those seen in patients with normal renal function receiving 400 mg/day. Dose reductions are necessary for patients with moderate and severe renal impairment [see Dosage and Administration (2.12) ] . Pediatric Use As in adult patients, imatinib was rapidly absorbed after oral administration in pediatric patients, with a C max of 2-4 hours. Apparent oral clearance was similar to adult values (11.0 L/hr/m 2 in children vs 10.0 L/hr/m 2 in adults), as was the half-life (14.8 hours in children vs 17.1 hours in adults). Dosing in children at both 260 mg/m 2 and 340 mg/m 2 achieved an AUC similar to the 400 mg dose in adults. The comparison of AUC on Day 8 vs Day 1 at 260 mg/m2 and 340 mg/m 2 dose levels revealed a 1.5- and 2.2-fold drug accumulation, respectively, after repeated once-daily dosing. Mean imatinib AUC did not increase proportionally with increasing dose. Based on pooled population pharmacokinetic analysis in pediatric patients with hematological disorders (CML, Ph+ ALL, or other hematological disorders treated with imatinib), clearance of imatinib increases with increasing BSA. After correcting for the BSA effect, other demographics, such as age, body weight, and body mass index did not have clinically significant effects on the exposure of imatinib. The analysis confirmed that exposure of imatinib in pediatric patients receiving 260 mg/m 2 once daily (not exceeding 400 mg once daily) or 340 mg/m 2 once daily (not exceeding 600 mg once daily) were similar to those in adult patients who received imatinib 400 mg or 600 mg once daily. Drug Interactions Agents Inducing CYP3A Metabolism Pretreatment of healthy volunteers with multiple doses of rifampin followed by a single dose of imatinib mesylate tablets, increased imatinib mesylate tablets oral-dose clearance by 3.8-fold, which significantly (p less than 0.05) decreased mean C max and AUC. Similar findings were observed in patients receiving 400 to 1200 mg/day imatinib mesylate tablets concomitantly with enzyme-inducing anti-epileptic drugs (EIAED) (e.g., carbamazepine, oxcarbamazepine, phenytoin, fosphenytoin, phenobarbital, and primidone). The mean dose normalized AUC for imatinib in the patients receiving EIAED’s decreased by 73% compared to patients not receiving EIAED. Concomitant administration of imatinib mesylate tablets and St. John’s Wort led to a 30% reduction in the AUC of imatinib. Consider alternative therapeutic agents with less enzyme induction potential in patients when rifampin or other CYP3A4 inducers are indicated. Imatinib mesylate tablets doses up to 1,200 mg/day (600 mg twice daily) have been given to patients receiving concomitant strong CYP3A4 inducers [see Dosage and Administration (2.12) ] . Agents Inhibiting CYP3A Metabolism There was a significant increase in exposure to imatinib (mean C max and AUC increased by 26% and 40%, respectively) in healthy subjects when imatinib mesylate tablets were coadministered with a single dose of ketoconazole (a CYP3A4 inhibitor). Caution is recommended when administering imatinib mesylate tablets with strong CYP3A4 inhibitors (e.g., ketoconazole, itraconazole, clarithromycin, atazanavir, indinavir, nefazodone, nelfinavir, ritonavir, saquinavir, telithromycin, and voriconazole). Grapefruit juice may also increase plasma concentrations of imatinib and should be avoided. Interactions with Drugs Metabolized by CYP3A4 Imatinib mesylate tablets increase the mean C max and AUC of simvastatin (CYP3A4 substrate) 2- and 3.5-fold, respectively, suggesting an inhibition of the CYP3A4 by imatinib mesylate tablets. Particular caution is recommended when administering imatinib mesylate tablets with CYP3A4 substrates that have a narrow therapeutic window (e.g., alfentanil, cyclosporine, diergotamine, ergotamine, fentanyl, pimozide, quinidine, sirolimus, or tacrolimus). Imatinib mesylate tablets will increase plasma concentration of other CYP3A4 metabolized drugs (e.g., triazolo-benzodiazepines, dihydropyridine calcium channel blockers, certain HMG-CoA reductase inhibitors, etc.). Because warfarin is metabolized by CYP2C9 and CYP3A4, patients who require anticoagulation should receive low-molecular weight or standard heparin instead of warfarin. Interactions with Drugs Metabolized by CYP2D6 Imatinib mesylate tablets increased the mean C max and AUC of metoprolol by approximately 23% suggesting that imatinib mesylate tablets have a weak inhibitory effect on CYP2D6-mediated metabolism. No dose adjustment is necessary, however, caution is recommended when administering imatinib mesylate tablets with CYP2D6 substrates that have a narrow therapeutic window. Interactions with Acetaminophen In vitro , imatinib mesylate tablets inhibit the acetaminophen O-glucuronidate pathway (K i 58.5 μM). Coadministration of imatinib mesylate tablets (400 mg/day for 8 days) with acetaminophen (1,000 mg single dose on Day 8) in patients with CML did not result in any changes in the pharmacokinetics of acetaminophen. Imatinib mesylate tablets pharmacokinetics were not altered in the presence of single-dose acetaminophen. There is no pharmacokinetic or safety data on the concomitant use of imatinib mesylate tablets at doses greater than 400 mg/day or the chronic use of concomitant acetaminophen and imatinib mesylate tablets.'

In [13]:
clinical_pharmacology['genomic_ner'][3]

[{'end': 96,
  'entity_group': 'GENETIC',
  'score': 0.9999901652336121,
  'start': 73,
  'word': 'protein - tyrosine kinase'},
 {'end': 148,
  'entity_group': 'GENETIC',
  'score': 0.9952441453933716,
  'start': 125,
  'word': 'BCR - ABL tyrosine kinase'},
 {'end': 191,
  'entity_group': 'GENETIC',
  'score': 0.9999716281890869,
  'start': 176,
  'word': 'tyrosine kinase'},
 {'end': 784,
  'entity_group': 'GENETIC',
  'score': 0.9998764991760254,
  'start': 759,
  'word': 'receptor tyrosine kinases'},
 {'end': 819,
  'entity_group': 'GENETIC',
  'score': 0.9999940991401672,
  'start': 789,
  'word': 'platelet - derived growth factor'},
 {'end': 825,
  'entity_group': 'GENETIC',
  'score': 0.9995359182357788,
  'start': 821,
  'word': 'PDGF'},
 {'end': 847,
  'entity_group': 'GENETIC',
  'score': 0.9999921917915344,
  'start': 831,
  'word': 'stem cell factor'},
 {'end': 852,
  'entity_group': 'GENETIC',
  'score': 0.9999366998672485,
  'start': 849,
  'word': 'SCF'},
 {'end': 860,
  '

## Attaching data to DGIdb
FDA data has now been pulled and tagged through NLP portion of pipeline. All that remains is to clean it up and join it with our original drug_apps dataframe. This will link *indication evidence items* with the original *application numbers* in dataframe format for computational digestion. Once merged, these evidence items can be summed, and presented to the clinician through something as simple as a print statement. Alternatively, some resource could absorb and make use of these statements for clinical interpretation.
  
NOTE: After running through the exercise of making this, I think using NER as the primary task is a little bit messy interpretation wise. I think it should probably be reframed as a question answering task. Output from the question answering task could be presented in a similar manner but I imagine would be way more understandable at a glance. 


In [14]:
def cleanup_ner(row):
    entities = []
    for entry in row:
        entities.append(entry['word'])

    return(entities)

def sum_indications(row):
    return(len(row))

In [15]:
ner_results = pd.DataFrame().assign(application_number=list(indications['application_number']), diseases_ner=indications['diseases_ner'])
ner_results[0:5]

,application_number,diseases_ner
0,ANDA206547,"[{'end': 178, 'entity_group': '0', 'score': 0...."
1,ANDA207323,"[{'end': 93, 'entity_group': '0', 'score': 0.9..."
2,ANDA207495,"[{'end': 187, 'entity_group': '0', 'score': 0...."
3,ANDA204644,"[{'end': 187, 'entity_group': '0', 'score': 0...."
4,ANDA207586,"[{'end': 180, 'entity_group': '0', 'score': 0...."


In [16]:
ner_results['diseases_clean'] = ner_results['diseases_ner'].apply(cleanup_ner)
ner_results['evidence_items'] = ner_results['diseases_clean'].apply(sum_indications)
ner_results[0:5]

,application_number,diseases_ner,diseases_clean,evidence_items
0,ANDA206547,"[{'end': 178, 'entity_group': '0', 'score': 0....",[1 INDICATIONS AND USAGE Imatinib mesylate is ...,54
1,ANDA207323,"[{'end': 93, 'entity_group': '0', 'score': 0.9...",[INDICATIONS Cisplatin Injection is indicated ...,15
2,ANDA207495,"[{'end': 187, 'entity_group': '0', 'score': 0....",[1 INDICATIONS AND USAGE Imatinib mesylate tab...,51
3,ANDA204644,"[{'end': 187, 'entity_group': '0', 'score': 0....",[1 INDICATIONS AND USAGE Imatinib mesylate tab...,51
4,ANDA207586,"[{'end': 180, 'entity_group': '0', 'score': 0....",[1 INDICATIONS AND USAGE Imatinib mesylate is ...,49


In [17]:
fda_evidence = pd.merge(drug_apps, ner_results, left_on='application',right_on='application_number', how='left')

fda_evidence = fda_evidence.drop(columns=['application_number','diseases_ner']).reset_index(drop=True)
fda_evidence = fda_evidence.rename(columns={'diseases_clean':'indication_evidence'})

fda_evidence[0:5]

,drug,application,description,indication_evidence,evidence_items
0,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...,[1 INDICATIONS AND USAGE Imatinib mesylate is ...,51
1,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...,[1 INDICATIONS AND USAGE Imatinib mesylate is ...,51
2,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...,[1 INDICATIONS AND USAGE Imatinib mesylate is ...,54
3,IMATINIB,ANDA079179,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...,[1 INDICATIONS AND USAGE Imatinib mesylate is ...,50
4,IMATINIB,ANDA079179,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...,[1 INDICATIONS AND USAGE Imatinib mesylate is ...,50


In [18]:
search_set[search_set['application_number']=='NDA203697'] # perhaps a date/update difference?

,brand_name,application_number,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
6,VAZALORE,NDA203697,NaN,Uses Uses For temporary relief of minor aches ...,NaN,NaN,WARNINGS Warnings Reye’s Syndrome: Children an...,NaN,NaN,Active Ingredient (in Each capsules) Purposes:...,NaN,ACTIVE iNGREDIENT Active Ingredient (in Each c...,Do not use I f you have ever had an allergic r...,NaN,NaN,NaN
13,VAZALORE,NDA203697,NaN,Uses For temporary relief of minor aches and p...,NaN,NaN,Warnings Warnings Reye’s Syndrome: Children an...,NaN,NaN,Purposes: Aspirin (NSAID)* 325mg __ Pain relie...,NaN,Active Ingredient (in Each capsules) Aspirin (...,Stop use and ask a doctor Stop use and ask a d...,NaN,NaN,NaN


In [31]:
# Encorafenib
entry_no = 24
print(f'All FDA evidence found for {fda_evidence["drug"][entry_no]} ({fda_evidence["application"][entry_no]}: {fda_evidence["description"][entry_no]}')
for evidence_item in fda_evidence['indication_evidence'][entry_no]:
    print(f'Indication Evidence: {evidence_item}')

All FDA evidence found for ENCORAFENIB (NDA210496: BRAFTOVI: 50MG Discontinued CAPSULE | BRAFTOVI: 75MG Prescription CAPSULE
Indication Evidence: 1 INDICATIONS AND USAGE BRAFTOVI is a kinase inhibitor indicated : • in combination with binimetinib, for the treatment of patients with unresectable or metastatic
Indication Evidence: melanoma
Indication Evidence: with a BRAF V600E or V600K mutation, as detected by an FDA - approved test. ( 1. 1, 2. 1 ) • in combination with cetuximab, for the treatment of adult patients with metastatic
Indication Evidence: colorectal cancer
Indication Evidence: (
Indication Evidence: CRC
Indication Evidence: ) with a BRAF V600E mutation, as detected by an FDA - approved test, after prior therapy. ( 1. 2, 2. 1 ) Limitations of Use BRAFTOVI is not indicated for treatment of patients with wild - type BRAF
Indication Evidence: melanoma
Indication Evidence: or wild - type BRAF
Indication Evidence: CRC
Indication Evidence: . ( 1. 3, 5. 2 ) 1. 1 BRAF V600E or V600

In [32]:
# Aspirin
entry_no = 25
print(f'All FDA evidence found for {fda_evidence["drug"][entry_no]} ({fda_evidence["application"][entry_no]}: {fda_evidence["description"][entry_no]}')
for evidence_item in fda_evidence['indication_evidence'][entry_no]:
    print(f'Indication Evidence: {evidence_item}')

All FDA evidence found for ASPIRIN (NDA203697: VAZALORE: 81MG Over-the-counter CAPSULE | VAZALORE: 325MG Over-the-counter CAPSULE
Indication Evidence: Uses Uses For temporary relief of minor
Indication Evidence: aches
Indication Evidence: and
Indication Evidence: pains
Indication Evidence: associated with :
Indication Evidence: headache
Indication Evidence: Backache
Indication Evidence: muscular
Indication Evidence: aches
Indication Evidence: a cold
Indication Evidence: toothache
Indication Evidence: minor
Indication Evidence: pains
Indication Evidence: of
Indication Evidence: arthritis
Indication Evidence: premenstrual and
Indication Evidence: menstrual cramps
Indication Evidence: temporarily redices fever ask your doctor about other uses of aspirin capsules


In [33]:
# Cisplatin
entry_no = 35
print(f'All FDA evidence found for {fda_evidence["drug"][entry_no]} ({fda_evidence["application"][entry_no]}: {fda_evidence["description"][entry_no]}')
for evidence_item in fda_evidence['indication_evidence'][entry_no]:
    print(f'Indication Evidence: {evidence_item}')

All FDA evidence found for CISPLATIN (ANDA207323: CISPLATIN: 1MG/ML Prescription INJECTABLE
Indication Evidence: INDICATIONS Cisplatin Injection is indicated as therapy to be employed as follows : Metastatic
Indication Evidence: Testicular Tumors
Indication Evidence: In established combination therapy with other approved chemotherapeutic agents in patients with metastatic
Indication Evidence: testicular tumors
Indication Evidence: who have already received appropriate surgical and / or radio therapeutic procedures. Metastatic
Indication Evidence: Ovarian Tumors
Indication Evidence: In established combination therapy with other approved chemotherapeutic agents in patients with metastatic
Indication Evidence: ovarian tumors
Indication Evidence: who have already received appropriate surgical and / or radiotherapeutic procedures. An established combination consists of cisplatin and cyclophosphamide. Cisplatin Injection, as a single agent, is indicated as secondary therapy in patients with 

In [34]:
# Imatinib
entry_no = 2
print(f'All FDA evidence found for {fda_evidence["drug"][entry_no]} ({fda_evidence["application"][entry_no]}: {fda_evidence["description"][entry_no]}')
for evidence_item in fda_evidence['indication_evidence'][entry_no]:
    print(f'Indication Evidence: {evidence_item}')

All FDA evidence found for IMATINIB (ANDA078340: IMATINIB MESYLATE: EQ 100MG BASE Prescription TABLET | IMATINIB MESYLATE: EQ 400MG BASE Prescription TABLET
Indication Evidence: 1 INDICATIONS AND USAGE Imatinib mesylate is a kinase inhibitor indicated for the treatment of : Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive
Indication Evidence: chronic myeloid leukemia
Indication Evidence: ( Ph +
Indication Evidence: CML
Indication Evidence: ) in chronic phase ( 1. 1 ) Patients with Philadelphia chromosome positive
Indication Evidence: chronic myeloid leukemia
Indication Evidence: ( Ph +
Indication Evidence: CML
Indication Evidence: ) in blast crisis ( BC ), accelerated phase ( AP ), or in chronic phase ( CP ) after failure of interferon - alpha therapy ( 1. 2 ) Adult patients with relapsed or refractory Philadelphia chromosome positive
Indication Evidence: acute lymphoblastic leukemia
Indication Evidence: ( Ph +
Indication Evidence: ALL
Indication Evide

## Scratch
Normalization for genetic/chemical entites from pharmacogenomic sections. Can the normalizer help filter out some relevancy? (not really sure about this but going to run the exp to see what happens)

In [36]:
clinical_pharmacology

Dataset({
    features: ['clinical_pharmacology', 'application_number', 'tokens', 'genomic_ner', 'chemical_ner', 'diseases_ner', 'merged_ner_groups'],
    num_rows: 36
})

In [38]:
clinical_pharmacology

ner_results = pd.DataFrame().assign(application_number=list(clinical_pharmacology['application_number']), genomics_ner=clinical_pharmacology['genomic_ner'])
ner_results[0:5]

ner_results['genomics_clean'] = ner_results['genomics_ner'].apply(cleanup_ner)
ner_results['evidence_items'] = ner_results['genomics_clean'].apply(sum_indications)
ner_results[0:5]

t_fda_evidence = pd.merge(drug_apps, ner_results, left_on='application',right_on='application_number', how='left')

t_fda_evidence = t_fda_evidence.drop(columns=['application_number','genomics_ner']).reset_index(drop=True)
t_fda_evidence = t_fda_evidence.rename(columns={'genomics_clean':'relevant_pharmacology_evidence'})

t_fda_evidence[0:5]

,drug,application,description,relevant_pharmacology_evidence,evidence_items
0,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...,"[protein - tyrosine kinase, BCR - ABL tyrosine...",13.0
1,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...,"[protein - tyrosine kinase, BCR - ABL tyrosine...",13.0
2,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...,"[protein - tyrosine kinase, BCR - ABL tyrosine...",17.0
3,IMATINIB,ANDA079179,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...,"[protein - tyrosine kinase, BCR - ABL tyrosine...",13.0
4,IMATINIB,ANDA079179,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...,"[protein - tyrosine kinase, BCR - ABL tyrosine...",13.0


In [48]:
# Imatinib
entry_no = 2
print(f'All FDA evidence found for {t_fda_evidence["drug"][entry_no]} ({t_fda_evidence["application"][entry_no]}: {t_fda_evidence["description"][entry_no]}')
for evidence_item in t_fda_evidence['relevant_pharmacology_evidence'][entry_no]:
    print(f'Genomic Evidence: {evidence_item}')

All FDA evidence found for IMATINIB (ANDA078340: IMATINIB MESYLATE: EQ 100MG BASE Prescription TABLET | IMATINIB MESYLATE: EQ 400MG BASE Prescription TABLET
Genomic Evidence: protein - tyrosine kinase
Genomic Evidence: BCR - ABL tyrosine kinase
Genomic Evidence: tyrosine kinase
Genomic Evidence: receptor tyrosine kinases
Genomic Evidence: platelet - derived growth factor
Genomic Evidence: PDGF
Genomic Evidence: stem cell factor
Genomic Evidence: SCF
Genomic Evidence: c - kit
Genomic Evidence: PDGF
Genomic Evidence: SCF
Genomic Evidence: albumin
Genomic Evidence: α1 - acid glycoprotein
Genomic Evidence: CYP3A4
Genomic Evidence: cytochrome P450 enzymes
Genomic Evidence: CYP1A2
Genomic Evidence: CYP2D6


In [49]:
# Encorafenib
entry_no = 24
print(f'All FDA evidence found for {t_fda_evidence["drug"][entry_no]} ({t_fda_evidence["application"][entry_no]}: {t_fda_evidence["description"][entry_no]}')
for evidence_item in t_fda_evidence['relevant_pharmacology_evidence'][entry_no]:
    print(f'Genomic Evidence: {evidence_item}')

All FDA evidence found for ENCORAFENIB (NDA210496: BRAFTOVI: 50MG Discontinued CAPSULE | BRAFTOVI: 75MG Prescription CAPSULE
Genomic Evidence: BRAF V600E
Genomic Evidence: BRAF
Genomic Evidence: CRAF
Genomic Evidence: BRAF gene
Genomic Evidence: BRAF V600E
Genomic Evidence: BRAF kinases
Genomic Evidence: JNK1
Genomic Evidence: JNK2
Genomic Evidence: JNK3
Genomic Evidence: LIMK1
Genomic Evidence: LIMK2
Genomic Evidence: MEK4
Genomic Evidence: STK36
Genomic Evidence: BRAF V600 E, D, and K mutations
Genomic Evidence: BRAF V600E
Genomic Evidence: RAF
Genomic Evidence: MEK
Genomic Evidence: ERK
Genomic Evidence: RAS
Genomic Evidence: RAF
Genomic Evidence: MEK
Genomic Evidence: ERK
Genomic Evidence: BRAF
Genomic Evidence: BRAF V600E
Genomic Evidence: BRAF V600E
Genomic Evidence: BRAF
Genomic Evidence: EGFR
Genomic Evidence: MAPK
Genomic Evidence: BRAF
Genomic Evidence: BRAF
Genomic Evidence: EGFR


In [50]:
# Aspirin --- NaN t_fda_evidence[25:26]
entry_no = 25
print(f'All FDA evidence found for {t_fda_evidence["drug"][entry_no]} ({t_fda_evidence["application"][entry_no]}: {t_fda_evidence["description"][entry_no]}')
for evidence_item in t_fda_evidence['relevant_pharmacology_evidence'][entry_no]:
    print(f'Genomic Evidence: {evidence_item}')

All FDA evidence found for ASPIRIN (NDA203697: VAZALORE: 81MG Over-the-counter CAPSULE | VAZALORE: 325MG Over-the-counter CAPSULE


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_31694/567591607.py:4 in <cell  │
│ line: 4>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_31694/567591607.py'           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'float' object is not iterable

In [51]:
# Cisplatin
entry_no = 35
print(f'All FDA evidence found for {t_fda_evidence["drug"][entry_no]} ({t_fda_evidence["application"][entry_no]}: {t_fda_evidence["description"][entry_no]}')
for evidence_item in t_fda_evidence['relevant_pharmacology_evidence'][entry_no]:
    print(f'Genomic Evidence: {evidence_item}')

All FDA evidence found for CISPLATIN (NDA018057: PLATINOL-AQ: 0.5MG/ML Discontinued INJECTABLE | PLATINOL: 10MG/VIAL Discontinued INJECTABLE | CISPLATIN: 1MG/ML Prescription INJECTABLE | CISPLATIN: 50MG/VIAL Prescription INJECTABLE
Genomic Evidence: albumin
Genomic Evidence: transferrin
Genomic Evidence: gamma globulin
Genomic Evidence: albumin
